In [27]:

import os

import pandas as pd


In [ ]:
#construccion ruta datos Bronce donde estan datos Bronce
bronce_path = os.path.join(
    os.curdir, "Data_Lake", "Bronce"
)

#construccion ruta datos Silver
silver_path = os.path.join(
    os.curdir, "Data_Lake", "Silver"
)

# Nombre del archivo parquet de entrada (donde datos originales)
bronce_file_name = "erp_bronce.parquet"

# Nombre del archivo parquet de salida (donde guardo datos procesados-limpios)
silver_file_name = "erp_silver.parquet"

# Ruta completa del archivo Bronce y Silver
bronce_file_path = os.path.join(
    bronce_path, bronce_file_name
)
silver_file_path = os.path.join(
    silver_path, silver_file_name
)

# lectura datos bronce (primeras filas)
bronce_erp_df = pd.read_parquet(bronce_file_path)
bronce_erp_df.head()


# Compruebo si la columna de id contiene nulos (si aperecieron)
bronce_erp_df[bronce_erp_df['RAZONSOCIAL'].isnull() == True]

#bronce_erp_df.info ()


# Borramos filas donde la columna es nula (ok) (eliminacion de 2 filas)
bronce_erp_df = bronce_erp_df.dropna(subset=["RAZONSOCIAL"])

bronce_erp_df[bronce_erp_df['RAZONSOCIAL'].isnull() == True]

bronce_erp_df.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 1951 entries, 0 to 1952
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   RAZONSOCIAL         1951 non-null   object        
 1   NOMBRECOMERCIAL     1255 non-null   object        
 2   DIRECCION           1941 non-null   object        
 3   SITIOWEB            1674 non-null   object        
 4   EMAIL               1836 non-null   object        
 5   TELEFONO            1810 non-null   object        
 6   TELEFONO2           97 non-null     object        
 7   FAX                 1500 non-null   object        
 8   PRODUCTOS           1944 non-null   object        
 9   EMPLEOTOTAL         204 non-null    float64       
 10  RANGOFACTURACION    1949 non-null   object        
 11  IMPORTA             525 non-null    object        
 12  EXPORTA             588 non-null    object        
 13  FECHAACTUALIZACION  1949 non-null   object        
 1